In [ ]:
import os
import csv
from paddleocr import PaddleOCR, draw_ocr
from paddleocr import PPStructure, save_structure_res
import preprocess_report as pre

# Path to your CSV file
csv_file_path = 'datasets/metadata for reports/training_meta_data - Blad1.csv'
ocr = PaddleOCR(use_angle_cls=True, lang='sv', use_gpu=True)
table_engine = PPStructure(layout=True, show_log=True, lang='en', ocr = False,use_gpu=True)  #consider layout, less found tbales but better strucutre?


with open(csv_file_path, mode='r', encoding='utf-8') as file:
   
    csv_reader = csv.DictReader(file)

  
    testindex = 0
    for row in csv_reader:
        if testindex == 10:
            break
        testindex += 1
        
        filename = row['FileName']
        approved = row['approved']
        result = row['resultat']
        balans = row['balans']

        if approved == 'Y':
            print(f'Filename: {filename}, resultat: {result}, balans: {balans}')

            result_pages = [] if result == "NA" else result.split('/')
            balans_pages = [] if balans == "NA" else balans.split('/')

            images = pre.convert_report_to_image(f"datasets/reports/{filename}")
            images = pre.crop_ndarrays_left_top_percentage(images)
            save_path = os.path.join('./images', filename)
            os.makedirs(save_path, exist_ok=True)
            resultImages = []
            resultImage = []
            for i in result_pages:
                resultImages.append(images[int(i)])
            if resultImages:

                imageIndex = 0
                for i in resultImages:
                    save_folder = f'./output/{filename}/'
                

                    file_path = os.path.join(save_path, f"resultat{imageIndex}.jpg")
                    #cv2.imwrite(file_path, i)

                    table = table_engine(i)
                    save_structure_res(table, save_folder, "result")
                    imageIndex += 1

            balansImages = []
            balansImage = []
            for i in balans_pages:
                balansImages.append(images[int(i)])

            if balansImages:
                imageIndex = 0
                for i in balansImages:
                    save_folder = f'./output/{filename}/'
                    img = balansImage
                    file_path = os.path.join(save_path, f"balans{imageIndex}.jpg")
                    #cv2.imwrite(file_path, i)
                    table = table_engine(i)
                    save_structure_res(table, save_folder, "balans")
                    imageIndex += 1

